## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [5]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

### PromptTemplate


In [6]:
from langchain.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [7]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [8]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [9]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

### Model(LLM)

`ChatOpenAI`

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://www.teddynote.com/openai/openai-models/
- 링크: https://platform.openai.com/docs/models


In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

### Output


In [11]:
from langchain_core.output_parsers import StrOutputParser

output = StrOutputParser()

## Chain 생성

**LCEL(LangChain Expression Language)**

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달되며, 그 다음 모델 출력은 출력 파서로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 정말로 이해할 수 있습니다.


In [12]:
# 주어진 토픽에 대한 짧은 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")
# OpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model="gpt-3.5-turbo-1106")
# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

# 완성된 Chain 을 이용하여 country 를 '대한민국'으로 설정하여 실행합니다.
chain.invoke({"country": "대한민국"})

'대한민국의 수도는 서울입니다.'

In [13]:
# 완성된 Chain 을 이용하여 country 를 '미국'으로 설정하여 실행합니다.
chain.invoke({"country": "미국"})

'미국의 수도는 워싱턴 D.C.입니다.'

## 전체 파이프라인

단계를 따라가려면:

1. 원하는 주제에 대한 사용자 입력을 `{"topic": "양자역학"}`로 전달합니다.
2. `prompt` 컴포넌트는 사용자 입력을 받아 `topic`을 사용하여 프롬프트를 구성한 후 PromptValue를 생성합니다.
3. `model` 컴포넌트는 생성된 프롬프트를 가져와 OpenAI LLM 모델에 평가를 위해 전달합니다. 모델에서 생성된 출력은 `ChatMessage` 객체입니다.
4. 마지막으로, `output_parser` 컴포넌트는 `ChatMessage`를 받아 이를 Python 문자열로 변환하며, 이는 invoke 메서드에서 반환됩니다.


```mermaid
graph LR
    A(입력: topic=양자역학) --> |Dict| B(PromptTemplate)
    B --> |PromptValue| C(ChatModel)
    C --> |ChatMessage| D(StrOutputParser)
    D --> |String| F(결과)
```


`prompt` 또는 `prompt | model`과 같은 체인의 작은 버전을 테스트하여 중간 결과를 확인하고자 하는 경우에 대해 유의하세요.


이 코드는 `prompt` 객체와 `model` 객체를 사용하여 대화형 AI 시스템을 시뮬레이션합니다. 먼저, `prompt.invoke` 메서드를 호출하여 사용자 입력에 대한 프롬프트 값을 생성합니다. 그런 다음, `prompt` 객체와 `model` 객체를 파이프 연산자(`|`)로 연결하고 `invoke` 메서드를 호출하여 AI 모델이 생성한 응답을 반환합니다. 이 과정에서 AI 모델은 'ice cream'이라는 주제에 대한 짧은 농담을 생성합니다.


In [14]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

# input 딕셔너리에 주제를 'ice cream'으로 설정합니다.
input = {"topic": "양자역학"}

# prompt 객체의 invoke 메서드를 사용하여 input을 전달하고 대화형 프롬프트 값을 생성합니다.
prompt.invoke(input)

# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
(prompt | model).invoke(input)

AIMessage(content='양자역학은 아주 작은 입자들인 원자나 입자들의 움직임과 상호작용을 다루는 물리 이론입니다. 양자역학은 이런 아주 작은 입자들이 가지는 특징과 행동을 이해하고 설명하는데 사용됩니다. 이론적으로, 양자역학은 확률적이고 불확실한 세계를 다루며, 매우 이상한 현상들을 설명하기도 합니다. 이론은 파동함수를 사용하여 입자의 위치나 운동상태를 예측하고 설명하며, 특히 원자나 입자들의 에너지 준위와 빛의 특성 등을 설명하는데 사용됩니다.양자역학은 현대 물리학의 중심적인 이론 중 하나이며, 많은 현상을 설명하고 예측하는데 사용됩니다.')

In [17]:
# parse_output 메서드를 사용하여 AI 모델이 생성한 메시지 문자열로 출력합니다.
(prompt | model | output_parser).invoke(input)

'양자역학은 아주 작은 입자들, 예를 들어 원자나 입자들의 움직임을 설명하는 물리 이론입니다. 양자역학에서는 입자들이 파동의 형태로도 행동할 수 있다고 가정합니다. 또한 입자의 위치나 운동 상태를 정확히 예측하는 것이 불가능하며, 확률적으로만 예측할 수 있다는 원리가 있습니다. 양자역학은 물리 현상을 설명하는 데 있어서 고전 물리학이나 상대성 이론으로는 설명할 수 없는 현상들을 다루기 위해 발전된 이론입니다.'